In [ ]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.linalg import null_space, svd
from numpy.linalg import solve, det, eigvals
import warnings

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, LabelSet

from ipywidgets import interact, FloatSlider

output_notebook(hide_banner=True)

# Bead on a hoop

A circular wire hoop rotates with constant angular velocity $\omega$ about a vertical diameter. A small bead
moves, with friction ($\alpha\neq 0$) or without friction ($\alpha=0$), along the hoop.

The equation of motion can be shown to be (using the standard notation in classical mechanics):

$$ \ddot{\theta} = -\omega_c^2 \sin \theta + \omega^2 \sin \theta \, \cos \theta - \alpha \theta $$

with $\omega_c = \sqrt{g/R}$, where the gravity acceleration is denoted by $g$ and the radius of the hoop is denoted $R$. The coefficient $\alpha$ is related to the friction in the system and can be idealized to be zero in the frictionless configuration.

Let $y_1=\theta$ and $y_2 = \dot{\theta}$. Then, we can switch to a first order system of differential equations:

$$
\left\{\begin{aligned}
{\mathrm d}_t y_1 & = y_2\\
{\mathrm d}_t y_2 & = (\omega^2 \cos y_1 - \omega_c^2)\sin y_1  - \alpha y_2
\end{aligned}\right.
$$


In [ ]:
class bead_hoop_friction_model:

    def __init__(self, omega, alpha):
        self.omega = omega
        self.alpha = alpha

    def fcn(self, t, y):
        y1, y2 = y
        omega = self.omega
        alpha = self.alpha
        omega_c = np.sqrt(9.81)
        y1_dot = y2
        y2_dot = np.sin(y1)*(omega*omega*np.cos(y1) - omega_c*omega_c) - alpha*y2
        return np.array([y1_dot, y2_dot])
    def jac_eq_1(self):
        omega = self.omega
        alpha = self.alpha
        omega_c = np.sqrt(9.81)
        j21 = omega*omega - omega_c*omega_c
        return np.array([[0., 1.], [j21 , -alpha]])

    def jac_eq_2(self):
        omega = self.omega
        alpha = self.alpha
        omega_c = np.sqrt(9.81)
        j21 = omega*omega + omega_c*omega_c
        return np.array([[0., 1.], [j21 , -alpha]])

    def jac_eq_3(self):
        omega = self.omega
        omega_c = np.sqrt(9.81)
        alpha = self.alpha
        
        if (omega<=omega_c):
            j21 = omega*omega - omega_c*omega_c
            return np.array([[0., 1.], [j21 , -alpha]])
        else:
            j21 = (omega_c**4)/(omega**2) - omega**2            
            return np.array([[0., 1.], [j21 , -alpha]])

In [ ]:
def plot_sol():

    yini = (1. , 0.)
    tini = 0.
    tend = 100.
    nt = 10001
    
    omega_c = np.sqrt(9.81)
    alpha = 0.5 #the case without friction corresponds to alpha=0
    
    bhfm = bead_hoop_friction_model(omega=2., alpha=alpha)
    fcn = bhfm.fcn 
    jac_eq_1 = bhfm.jac_eq_1
    jac_eq_2 = bhfm.jac_eq_2
    jac_eq_3 = bhfm.jac_eq_3
        
    tol = 1.e-10
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    eig_vals_eq_1, _= np.linalg.eig(jac_eq_1())
    eig_vals_eq_2, _= np.linalg.eig(jac_eq_2())
    eig_vals_eq_3, _= np.linalg.eig(jac_eq_3())

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_y1 = fig_sol.line(sol.t, sol.y[0], legend_label="y1", line_width=2)    
    plt_y2 = fig_sol.line(sol.t, sol.y[1], legend_label="y2", line_width=2, color="green")
    
    fig_pha = figure(plot_height=300, plot_width=475, title="Phase portrait")
    plt_pha = fig_pha.line(sol.y[0], sol.y[1], line_width=2)
    
    fig_eig = figure(x_range=(-6,6), y_range=(-3.5,3.5), plot_height=300, plot_width=475, 
                     title="Eigenvalues at equilibrium points")
    fig_eig.line(x=np.linspace(-6.,6,10), y=0, color="black")
    fig_eig.line(x=0, y=np.linspace(-3.5,3.5,10), color="black")
    plt_eig_3 = fig_eig.x(np.real(eig_vals_eq_3), np.imag(eig_vals_eq_3), line_width=2, size=10, color="cornflowerblue",
                               legend_label="Equilibrium 3")
    plt_eig_2 = fig_eig.x(np.real(eig_vals_eq_2), np.imag(eig_vals_eq_2), line_width=2, size=10, color="green",
                               legend_label="Equilibrium 2")
    plt_eig_1 = fig_eig.x(np.real(eig_vals_eq_1), np.imag(eig_vals_eq_1), line_width=2, size=10, color="crimson",
                               legend_label="Equilibrium 1")


    show(column(fig_sol, row(fig_pha, fig_eig)), notebook_handle=True)
    
    def update(omega):
        bhfm = bead_hoop_friction_model(omega, alpha=alpha)
        fcn = bhfm.fcn 
        jac_eq_1 = bhfm.jac_eq_1
        jac_eq_2 = bhfm.jac_eq_2
        jac_eq_3 = bhfm.jac_eq_3
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        eig_vals_eq_1, _= np.linalg.eig(jac_eq_1())
        eig_vals_eq_2, _= np.linalg.eig(jac_eq_2())
        eig_vals_eq_3, _= np.linalg.eig(jac_eq_3())

        plt_y1.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_y2.data_source.data = dict(x=sol.t, y=sol.y[1])
        plt_pha.data_source.data = dict(x=sol.y[0], y=sol.y[1])
        plt_eig_1.data_source.data = dict(x=np.real(eig_vals_eq_1), y=np.imag(eig_vals_eq_1))
        plt_eig_2.data_source.data = dict(x=np.real(eig_vals_eq_2), y=np.imag(eig_vals_eq_2))
        plt_eig_3.data_source.data = dict(x=np.real(eig_vals_eq_3), y=np.imag(eig_vals_eq_3))
        push_notebook()

    min = omega_c - 3.13
    max = omega_c + 3.13
    step = (max-min)/10000
    interact(update, omega=FloatSlider(min=min, max=max, value=omega_c, step=step, continuous_update=False))
    
plot_sol()

## Pitchfork bifurcation 

In [ ]:
def continuation_pitchfork(f, df, pt0, vect0, ds, depth, nb_pts_max=1e3, tol=1e-10, it_max=20):
    #### data structure: variables pt0, pt1, etc are of the form [lambda, x], where lambda is the varied paramter and x a corresponding equilibria    
    
    print("Current depth: ", depth)
    
    tab_sol = pt0[...,None] ## table where we store the equilibria (+ corresponding parameter value)
    tab_bif = np.array([]).reshape(3,0) ## table where we store the equilibria (+ corresponding parameter value) at which a pitchfork occurs
    ev = eigvals(df(pt0)[:,1:]) 
    stab = np.array([np.all( np.real(ev) < 0 )])
    tab_stab = stab ## table where we store the stability of the equilibria stored in tab_sol
    
    
    def Newton(pt, p0, v0, tol, it_max):
        ## Newton method for the enlarged map containing f + the pseudo arclength continuation equation
        res = np.concatenate((np.array([np.dot(pt-p0,v0)]),f(pt)))
        err = np.sum(np.abs(res))
        it = 0
        while (err > tol) & (it < it_max):
            pt = pt - solve(np.block([[v0],[df(pt)]]), res)
            res = np.concatenate((np.array([np.dot(pt-p0,v0)]),f(pt)))
            err = np.sum(np.abs(res))
            it = it + 1
        if err > tol:
            warnings.warn("Newton may not have converged")
            print(err)
        return pt
         
    if len(vect0) == 0:
        ## if there is no initial tangent vector given, we compute one
        vect0 = null_space(df(pt0))
        if vect0.shape[1] > 1:
            warnings.warn("The starting point seems to be singular")
        vect0 = vect0[:,0]
    
    df0 = df(pt0)
    det0 = det(np.block([[vect0],[df0]]))
    nb_pts = 0
    while nb_pts < nb_pts_max:
        pt1_pred = pt0 + ds * vect0 ## predictor
        pt1 = Newton(pt1_pred, pt1_pred, vect0, tol, it_max) ##corrector using Newton's method
        df1 = df(pt1)
        vect1 = svd(df1)[2][-1,:] ## computation of a new tangent vector at the new point pt1
        if np.dot(vect1,vect0) < 0: ## checking that we still go in the same direction
            vect1 = - vect1
        det1 = det(np.block([[vect1],[df1]]))
        if det0*det1 > 0: ## no bifurcation, we store the newly computed point and keep going
            tab_sol = np.concatenate((tab_sol, pt1[...,None]), axis=1)
            ev = eigvals(df(pt1)[:,1:])
            stab = np.array([np.all( np.real(ev) < 0 )])
            tab_stab = np.concatenate((tab_stab, stab))
            nb_pts = nb_pts + 1
            pt0 = pt1
            vect0 = vect1
            df0 = df1
            det0 = det1
        else: ## bifurcation
            print("Pitchfork detected")
            tab_bif = np.concatenate((tab_bif, pt0[...,None]), axis=1)
            if depth > 0: ## we compute the bifurcating direction and restart on each branch
                V = svd(df0)[2] ## the last rows of V should approximately span the kernel of the Jacobian df0
                vect_cont = V[-1,:] 
                vect_bif = V[-2,:]
                if np.abs(np.dot(vect_cont,vect0)) < np.abs(np.dot(vect_bif,vect0)): ## checking that we correctly identified which vector is continuing the original branch and which one is bifurcating
                    vect_temp = vect_cont
                    vect_cont = vect_bif
                    vect_bif = vect_temp
                if np.dot(vect_cont,vect0) < 0:
                    vect_cont = -vect_cont ## checking that the continuation vector still goes in the same direction
                
                pt10_pred = pt0 + ds * vect_cont ## predictor on the original branch
                pt10 = Newton(pt10_pred, pt10_pred, vect_cont, tol, it_max) ## corrector on the original branch
                tab_sol_10, tab_bif_10, tab_stab_10 = continuation_pitchfork(f, df, pt10, vect_cont, ds, depth-1, nb_pts_max, tol, it_max) ## continuation on the original branch
                
                pt11_pred = pt0 + ds * vect_bif ## predictor on one side of the bifurcating branch
                pt11 = Newton(pt11_pred, pt11_pred, vect_bif, tol, it_max) ## corrector on that side of the bifurcating branch
                tab_sol_11, tab_bif_11, tab_stab_11 = continuation_pitchfork(f, df, pt11, vect_bif, ds, depth-1, nb_pts_max, tol, it_max) ## continuation on that side of the bifurcating branch
                
                pt12_pred = pt0 - ds * vect_bif ## predictor on the other side of the bifurcating branch
                pt12 = Newton(pt12_pred, pt12_pred, -vect_bif, tol, it_max) ## corrector on that other side of the bifurcating branch
                tab_sol_12, tab_bif_12, tab_stab_12 = continuation_pitchfork(f, df, pt12, -vect_bif, ds, depth-1, nb_pts_max, tol, it_max) ## continuation on that other side of the bifurcating branch
                
                tab_sol = np.concatenate((tab_sol, tab_sol_10, tab_sol_11, tab_sol_12), axis=1)
                tab_bif = np.concatenate((tab_bif, tab_bif_10, tab_bif_11, tab_bif_12), axis=1)
                tab_stab = np.concatenate((tab_stab, tab_stab_10, tab_stab_11, tab_stab_12))
            return tab_sol, tab_bif, tab_stab 
    print("Maximal number of points reached on the current branch")
    return tab_sol, tab_bif, tab_stab

In [ ]:
omega_c = np.sqrt(9.81)
alpha = 0.5

def f(pt):
    omega, y1, y2 = pt
    return np.array([y2, (omega**2*np.cos(y1)-omega_c**2) * np.sin(y1) - alpha*y2])

def df(pt):
    omega, y1, y2 = pt
    return np.block([[np.array([0, 0, 1])],[np.array([omega*np.sin(2*y1), -omega**2*np.sin(y1)**2+(omega**2*np.cos(y1)-omega_c**2)*np.cos(y1), -alpha])]])


tab_sol, tab_bif, tab_stab = continuation_pitchfork(f, df, pt0 = np.array([0,0,0]), vect0 = np.array([1,0,0]), ds=1e-2, depth=2, nb_pts_max=1000)
# print(tab_sol)
# print(tab_bif)
# print(tab_stab)

fig_branch = figure(plot_height=500, plot_width=950)
fig_branch.x(tab_sol[0, tab_stab], tab_sol[1, tab_stab], legend_label="stable equilibria", color="green")
fig_branch.x(tab_sol[0, np.logical_not(tab_stab)], tab_sol[1, np.logical_not(tab_stab)], legend_label="unstable equilibria", color="red")
fig_branch.x(tab_bif[0, :], tab_bif[1, :], size = 10, legend_label="bifurcation", color="black")
fig_branch.legend.location = "top_left"
show(fig_branch)